In [65]:
import pandas as pd 

In [66]:
ds_raw = pd.read_json('D:/work_dir/Datasets/LoRa_anomaly-detection/RSSI_fingerprinting/data/lorawan_antwerp_2019_dataset.json')
ds = pd.read_csv('D:/work_dir/Datasets/LoRa_anomaly-detection/RSSI_fingerprinting/files/x_weather.csv', index_col=0)

In [117]:
# Create a list to collect updates
updates = []

# Loop over each reference and gather data
for ref in ds['gw_info_row_id']:
    gw_meta = ds_raw.loc[ref, 'gateways']  # Get the gateways data for the ref
    for gw in gw_meta:
        updates.append((ref, gw['id'], gw['rx_time']['time']))

# Convert to DataFrame
updates_df = pd.DataFrame(updates, columns=['ref', 'gw_id', 'rx_time'])

# Pivot to create the proper structure in one step
toa_df = updates_df.pivot(index='ref', columns='gw_id', values='rx_time')



In [68]:
result = pd.merge(ds, toa_df, left_on='gw_info_row_id', right_index=True)

In [69]:
toa_df.shape[1]

44

In [70]:
toa_df.columns[1]

'004A05D4'

In [118]:
pd.Timestamp(toa_df.iloc[1, 6]).value

1546588793079000000

In [119]:
# Convert all columns to datetime format
for col in toa_df.columns:
    toa_df[col] = pd.to_datetime(toa_df[col])


In [120]:
type(toa_df.iloc[1, 6])

pandas._libs.tslibs.timestamps.Timestamp

In [121]:
toa_df

gw_id,004A026B,004A05D4,004A0D9C,004A0DA4,080605EE,08060622,08060716,080E00B9,080E0116,080E0149,...,FF010877,FF010894,FF0108A3,FF010930,FF010957,FF01099B,FF010EFE,FF01753E,FF017649,FF0178DF
ref,,,,,,,,,,,,,,,,,,,,,
1,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:56.196000+01:00,NaT,NaT,NaT,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:56.786565370+01:00,NaT,2019-01-04 08:59:56.786529370+01:00
2,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:53.079000+01:00,NaT,NaT,NaT,...,NaT,NaT,2019-01-04 08:59:53.962028334+01:00,NaT,NaT,NaT,NaT,2019-01-04 08:59:53.962030483+01:00,NaT,2019-01-04 08:59:53.962029179+01:00
4,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:39.406000+01:00,NaT,NaT,NaT,...,NaT,NaT,2019-01-04 08:59:39.200708874+01:00,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:39.200711152+01:00
5,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:38.004000+01:00,...,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:38.892894509+01:00,NaT,NaT,NaT
6,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:34.595000+01:00,2019-01-04 08:59:34.594000+01:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-04 08:59:34.992718108+01:00,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130368,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-14 15:12:19.722000+01:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
130372,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-14 15:11:48.419000+01:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT
130375,NaT,NaT,NaT,NaT,2019-01-14 15:11:28.516000+01:00,2019-01-14 15:11:28.517000+01:00,2019-01-14 15:11:28.517000+01:00,NaT,NaT,2019-01-14 15:11:28.516000+01:00,...,NaT,NaT,NaT,NaT,NaT,NaT,NaT,NaT,2019-01-14 15:11:28.312846368+01:00,NaT


In [124]:
from itertools import combinations

tdoa_df = pd.DataFrame()

for i, j in combinations(range(toa_df.shape[1]), 2):
    col_name = f'{toa_df.columns[i]} - {toa_df.columns[j]}'
    # Calculate the difference in seconds. Automatically handling NaN
    diff = abs((toa_df.iloc[:, i] - toa_df.iloc[:, j]).dt.total_seconds())
    
    tdoa_df[col_name] = diff

C:\Users\Sudip Barua\AppData\Local\Temp\ipykernel_3728\2935674565.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdoa_df[col_name] = diff
C:\Users\Sudip Barua\AppData\Local\Temp\ipykernel_3728\2935674565.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  tdoa_df[col_name] = diff
C:\Users\Sudip Barua\AppData\Local\Temp\ipykernel_3728\2935674565.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all colum